# Классификация изображений Тома и Джерри

Ссылка на датасет - [Kaggle](https://www.kaggle.com/datasets/balabaskar/tom-and-jerry-image-classification)

## Задача:
1. Обучть модель классифицировать изображения на два класса

![](https://storage.googleapis.com/kaggle-datasets-images/2136537/3554162/0473eccb7645c78ad29d97fcf69a3127/dataset-cover.jpg?t=2022-04-30-12-38-18)

## Используемые библиотеки

In [15]:
import keras, shutil
import numpy as np
import pandas as pd
from keras import layers
from keras import Sequential
from keras import regularizers
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

## Константы

In [22]:
BATCH_SIZE = 20
BASE_PATH = os.path.join(os.getcwd(), '..', '..', 'data', 'tom-and-jerry')
TRAIN_PATH = os.path.join(BASE_PATH, 'train')
VAL_PATH = os.path.join(BASE_PATH, 'val')
TEST_PATH = os.path.join(BASE_PATH, 'test')

## Загрузка данных


<span style="color:red">Добавить позже инстуркцию по загрузке данных</span>.

### Разделение данных

In [33]:
PARAMS = {
    'samples': [
        {
            'train': 700
        },
        {
            'val': 240
        },
        {
            'test': 240
        }
    ],
    'classes': ['tom', 'jerry']
}

In [34]:
print('Количество картинок с Томом: ', len(os.listdir(os.path.join(BASE_PATH, 'src', 'tom'))))
print('Количество картинок с Джерри', len(os.listdir(os.path.join(BASE_PATH, 'src', 'jerry'))))

Количество картинок с Томом:  1930
Количество картинок с Джерри 1240


In [35]:
for tag in PARAMS['classes']:
    paths = os.listdir(os.path.join(BASE_PATH, 'src', tag))

    for sample in PARAMS['samples']:
        name, count = list(sample.items())[0]

        shutil.rmtree(os.path.join(BASE_PATH, name, tag),  ignore_errors=True)
        os.makedirs(os.path.join(BASE_PATH, name, tag))

        for path in paths[:count]:
            shutil.copy(os.path.join(BASE_PATH, 'src', tag,  path), os.path.join(BASE_PATH, name, tag))
        
        paths = paths[count:]

### Создание ImageDataGenerator

In [57]:
train_datagen = ImageDataGenerator(
    rescale=1/255,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True
)
# test_datagen = ImageDataGenerator(rescale=1/255)

train_generator = train_datagen.flow_from_directory(
    TRAIN_PATH,
    target_size=(150, 150),
    batch_size=BATCH_SIZE,
    class_mode='binary'
)
val_generator = train_datagen.flow_from_directory(
    VAL_PATH,
    target_size=(150, 150),
    batch_size=BATCH_SIZE,
    class_mode='binary'
)

Found 1400 images belonging to 2 classes.
Found 480 images belonging to 2 classes.


## Конструирование сети

In [66]:
model = Sequential()

# Сверточная основа
model.add(layers.Conv2D(16, (3, 3), activation='relu', input_shape=(150, 150, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(32, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Полносвязный классификатор
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

# Настройка оптимизаторов и метрик
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['acc'])

In [67]:
history = model.fit(
    train_generator,
    batch_size=BATCH_SIZE,
    epochs=120,
    steps_per_epoch=700 / BATCH_SIZE,
    validation_data=val_generator,
    validation_steps=240 / BATCH_SIZE,
)

Epoch 1/120
35/35 [==============================] - 21s 580ms/step - loss: 0.7795 - acc: 0.5457 - val_loss: 0.6845 - val_acc: 0.5083
Epoch 2/120
35/35 [==============================] - 20s 576ms/step - loss: 0.6636 - acc: 0.6057 - val_loss: 0.6367 - val_acc: 0.6417
Epoch 3/120
35/35 [==============================] - 20s 584ms/step - loss: 0.5898 - acc: 0.6986 - val_loss: 0.5822 - val_acc: 0.7125
Epoch 4/120
35/35 [==============================] - 20s 556ms/step - loss: 0.5294 - acc: 0.7329 - val_loss: 0.5196 - val_acc: 0.7375
Epoch 5/120
35/35 [==============================] - 24s 691ms/step - loss: 0.5224 - acc: 0.7629 - val_loss: 0.5594 - val_acc: 0.7250
Epoch 6/120
35/35 [==============================] - 20s 565ms/step - loss: 0.4424 - acc: 0.7929 - val_loss: 0.4573 - val_acc: 0.7958
Epoch 7/120
35/35 [==============================] - 21s 610ms/step - loss: 0.4227 - acc: 0.8071 - val_loss: 0.5890 - val_acc: 0.7167
Epoch 8/120
35/35 [==============================] - 21s 595ms

In [68]:
model.save('tom-and-jerry.h5')